In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mip import Model, xsum, minimize, BINARY

# Set parameters
n_hours = 8760  # Number of hours in a year
wind_capacity = 4  # MW
wind_mean = 200  # MWh/h
wind_stddev = 100  # MWh/h
price_mean = 30  # EUR/MWh
price_stddev = 10  # EUR/MWh

# Generate random data for wind generation and price
np.random.seed(1)  # Set random seed for reproducibility
wind_gen = np.random.normal(wind_mean, wind_stddev, n_hours)
wind_gen[wind_gen < 0] = 0  # Ensure that wind generation is non-negative
price = np.random.normal(price_mean, price_stddev, n_hours)

# Define optimization model
model = Model()

# Define variables
s = [model.add_var(lb=0) for i in range(n_hours)]  # Storage level at each hour
charge = [model.add_var(lb=0) for i in range(n_hours)]  # Energy charged to storage at each hour
discharge = [model.add_var(lb=0) for i in range(n_hours)]  # Energy discharged from storage at each hour]

# Define constraints
model += s[0] == 0  # Storage level at the beginning of the year
for i in range(n_hours):
    model += s[i] <= wind_capacity + charge[i]  # Storage level cannot exceed capacity
    model += s[i] >= discharge[i]  # Storage level cannot go below zero
    if i < n_hours-1:
        model += s[i+1] == s[i] + charge[i] - discharge[i]  # Storage level at next hour

# Define objective function
model.objective = minimize(xsum(price[i]*(wind_gen[i] - discharge[i]) for i in range(n_hours)))

# Solve the optimization problem
model.optimize()

# Print the optimal revenue and storage capacity
if model.status == 0:
    revenue = round(model.objective_value, 2)
    storage_capacity = round(max([s[i].x for i in range(n_hours)]), 2)
    print("Optimal revenue: EUR", revenue)
    print("Storage capacity required: ", storage_capacity, "MWh")

    # Plot wind generation, energy charged, energy discharged, and storage level over time
    t = np.arange(n_hours)
    fig, ax = plt.subplots()
    ax.plot(t, wind_gen, label='Wind generation')
    ax.plot(t, [charge[i].x for i in range(n_hours)], label='Energy charged')
    ax.plot(t, [discharge[i].x for i in range(n_hours)], label='Energy discharged')
    ax.plot(t, [s[i].x for i in range(n_hours)], label='Storage level')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Energy (MWh)')
    ax.legend()
    plt.show()
